In [ ]:
# Hücre 1: Gerekli Kütüphanelerin Kurulumu (Wav2Vec2 için)

# Lütfen bu hücreyi çalıştırmadan önce, eğer yeni bir not defteri değilse,
# Colab çalışma zamanını "Disconnect and delete runtime" ile sıfırlamanız önerilir.
# Eğer yeni bir not defteri ise doğrudan çalıştırabilirsiniz.

print("Wav2Vec2 projesi için gerekli kütüphaneler kuruluyor...")

# 1. Başarılı BERT projesindeki stabil versiyonları hedefleyelim:
#    NumPy'ye dokunmuyoruz, Colab'ın varsayılanını kullanacağız.
#    Pandas'a dokunmuyoruz, Colab'ın varsayılanını kullanacağız.

# Datasets ve Transformers için bilinen iyi versiyonlar:
!pip install datasets==3.6.0 -q
print("Datasets 3.6.0 kuruldu.")

!pip install transformers==4.48.3 -q
print("Transformers 4.48.3 kuruldu.")

# 2. Ses işleme kütüphaneleri:
# torchaudio genellikle PyTorch ile uyumlu gelir, versiyon belirtmeden deneyelim.
# Colab'da PyTorch genellikle önceden kuruludur.
# Eğer PyTorch'unuzla uyumsuzluk olursa, PyTorch versiyonunuza uygun bir torchaudio kurmanız gerekebilir.
!pip install torchaudio -q
print("torchaudio kuruldu/güncellendi.")

!pip install librosa -q
print("librosa kuruldu/güncellendi.")

# 3. Diğer yardımcı kütüphaneler
!pip install scikit-learn matplotlib seaborn -q
print("Scikit-learn, Matplotlib, Seaborn kuruldu/güncellendi.")

print("\nKütüphane kurulumları (Hücre 1) tamamlandı.")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
print("!!! LÜTFEN ŞİMDİ Colab Çalışma Zamanını (Runtime -> Restart runtime)      !!!")
print("!!! KESİNLİKLE YENİDEN BAŞLATIN. Bu, kurulumların oturmasını sağlar.         !!!")
print("!!! Yeniden başlattıktan sonra Hücre 1'i TEKRAR ÇALIŞTIRMAYIN,           !!!")
print("!!! doğrudan Hücre 2'ye (Kütüphane Importları) geçin.                   !!!")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [ ]:
# Hücre 2: Kütüphanelerin Import Edilmesi, Versiyon Kontrolü ve Cihaz Belirleme (Wav2Vec2 için)

# Temel ve Hugging Face kütüphanelerinin import edilmesi
import transformers
import datasets
import torch
import torchaudio # Ses işleme için PyTorch kütüphanesi
import librosa    # Ses analizi ve işleme için
import numpy as np
import pandas as pd
import sklearn # scikit-learn'ün ana modülü
import matplotlib
import seaborn as sns
import sys    # Python versiyonunu almak için
import time   # Süre ölçümleri için

# Hugging Face kütüphanelerinden sık kullanılacak modüller (ilerleyen aşamalarda)
# from datasets import load_dataset # Zaten yukarıda var
# from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
# from transformers import TrainingArguments, Trainer

# Cihazı belirleme: GPU varsa GPU, yoksa CPU kullanılacak.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# İsteğe bağlı: Daha temiz bir çıktı için uyarıları bastırma
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print("Kütüphaneler başarıyla import edildi.")
print("-" * 50)
print("KULLANILAN KÜTÜPHANE VERSİYONLARI:")
print(f"  Python Versiyonu (sys.version): {sys.version.split()[0]}")
print(f"  PyTorch: {torch.__version__}")
if hasattr(torchaudio, '__version__'): # torchaudio bazen __version__ göstermeyebilir
    print(f"  Torchaudio: {torchaudio.__version__}")
else:
    print("  Torchaudio: Versiyon bilgisi alınamadı (ama import edildi).")
print(f"  Librosa: {librosa.__version__}")
print(f"  Transformers: {transformers.__version__}")
print(f"  Datasets: {datasets.__version__}")
print(f"  Numpy: {np.__version__}")
print(f"  Pandas: {pd.__version__}")
print(f"  Scikit-learn: {sklearn.__version__}")
print(f"  Matplotlib: {matplotlib.__version__}")
print(f"  Seaborn: {sns.__version__}")
# fsspec gibi alt bağımlılıkları da isterseniz yazdırabiliriz ama şimdilik bunlar yeterli.
print("-" * 50)
print(f"KULLANILACAK CİHAZ: {device}")
print("-" * 50)

if device.type == 'cuda':
    print(f"  CUDA Versiyonu (torch.version.cuda): {torch.version.cuda}")
    print(f"  cuDNN Versiyonu (torch.backends.cudnn.version()): {torch.backends.cudnn.version()}")
    print(f"  Kullanılabilir GPU Sayısı: {torch.cuda.device_count()}")
    if torch.cuda.device_count() > 0:
        print(f"  Aktif GPU Adı: {torch.cuda.get_device_name(0)}")
    print("-" * 50)
else:
    print("Uyarı: GPU bulunamadı veya aktif değil. Model eğitimi CPU üzerinde daha yavaş olacaktır.")
    print("Colab'da GPU'yu aktifleştirmek için 'Runtime' -> 'Change runtime type' menüsünü kullanabilirsiniz.")
    print("-" * 50)

In [ ]:
# Hücre 3: Ses Veri Setinin Yüklenmesi (SUPERB ks Örneği) ve Temel İnceleme (trust_remote_code ile Güncellenmiş)

from datasets import load_dataset
from IPython.display import Audio # Sesi not defterinde çalmak için
import random # Rastgele örnek seçmek için

# Örnek olarak SUPERB benchmark'ının Keyword Spotting ("ks") görevini kullanacağız.
dataset_name = "superb"
config_name = "ks" # Keyword Spotting

print(f"'{dataset_name}' veri setinin '{config_name}' yapılandırması yükleniyor (trust_remote_code=True)...")
try:
    # dataset_raw'ı global yapalım ki sonraki hücrelerden erişilebilsin
    global dataset_raw
    dataset_raw = load_dataset(dataset_name, config_name, trust_remote_code=True) # <<-- trust_remote_code=True eklendi

    print(f"\n'{dataset_name}' ({config_name}) veri seti başarıyla yüklendi.")
    print("\nYüklenen Ham Veri Seti Yapısı:")
    print(dataset_raw)

    available_splits = list(dataset_raw.keys())
    if not available_splits:
        raise ValueError("Veri setinde hiçbir alt küme (split) bulunamadı.")

    split_to_inspect = 'train' if 'train' in available_splits else available_splits[0]

    print(f"\n'{split_to_inspect}' alt kümesinin özellikleri (features):")
    print(dataset_raw[split_to_inspect].features)

    # Etiket bilgisini alalım (label_feature'ı global yapalım)
    global label_feature
    if 'label' in dataset_raw[split_to_inspect].features:
        label_feature = dataset_raw[split_to_inspect].features['label']
        print(f"\nEtiket Bilgisi ('label' özelliği):")
        print(label_feature)
        if hasattr(label_feature, 'names'):
            print(f"Toplam etiket sayısı: {label_feature.num_classes}")
            print(f"Etiket isimleri: {label_feature.names}")
        else:
            print("Uyarı: 'label' özelliği bir ClassLabel değil gibi görünüyor veya 'names' attribute'u yok.")
    else:
        print("Uyarı: Veri setinde 'label' özelliği bulunamadı.")
        label_feature = None

    if split_to_inspect in dataset_raw and len(dataset_raw[split_to_inspect]) > 0:
        print(f"\n'{split_to_inspect}' setinden rastgele bir ses örneği dinleniyor...")
        random_index = random.randint(0, len(dataset_raw[split_to_inspect]) - 1)
        random_sample = dataset_raw[split_to_inspect][random_index]

        # Ses verisi ve örnekleme oranını al
        # 'audio' özelliğinin varlığını ve beklenen yapıda olduğunu kontrol et
        if "audio" in random_sample and isinstance(random_sample["audio"], dict) and \
           "array" in random_sample["audio"] and "sampling_rate" in random_sample["audio"]:

            audio_data = random_sample["audio"]["array"]
            sampling_rate = random_sample["audio"]["sampling_rate"]

            print(f"  Örnek ID: {random_sample.get('file', 'N/A') if isinstance(random_sample, dict) else random_index}")
            print(f"  Örnekleme Oranı (Sampling Rate): {sampling_rate} Hz")

            if label_feature and 'label' in random_sample and hasattr(label_feature, 'int2str'):
                label_id = random_sample["label"]
                print(f"  Etiket ID: {label_id}, Etiket Adı: {label_feature.int2str(label_id)}")
            elif 'label' in random_sample:
                 print(f"  Etiket ID: {random_sample['label']}")

            display(Audio(data=audio_data, rate=sampling_rate))
        else:
            print("  Hata: Ses verisi veya örnekleme oranı seçilen örnekte beklenen formatta bulunamadı.")
            print(f"  Örnek içeriği: {random_sample}")

    else:
        print(f"\n'{split_to_inspect}' seti boş veya bulunamadı, örnek ses dinlenemiyor.")

except Exception as e:
    print(f"\nVeri seti yüklenirken veya işlenirken bir hata oluştu: {e}")
    import traceback
    print(traceback.format_exc())
    if 'dataset_raw' in globals(): del dataset_raw # Hata durumunda değişkeni temizle
    if 'label_feature' in globals(): del label_feature # Hata durumunda değişkeni temizle

In [ ]:
# Hücre 4: Ses Veri Seti Ön İşleme (Wav2Vec2 için - Düzeltilmiş max_length ile)

from transformers import AutoFeatureExtractor
import torchaudio
import torchaudio.transforms as T
# import librosa # Alternatif yeniden örnekleme için, gerekirse aktif edilebilir

# dataset_raw ve label_feature'ın Hücre 3'te tanımlandığını varsayıyoruz.
model_checkpoint_w2v2 = "facebook/wav2vec2-base"

if 'dataset_raw' in globals() and dataset_raw is not None:
    try:
        print(f"'{model_checkpoint_w2v2}' için Feature Extractor yükleniyor...")
        global feature_extractor # Diğer hücrelerde gerekebilir
        feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint_w2v2)

        target_sampling_rate = feature_extractor.sampling_rate
        print(f"Feature extractor başarıyla yüklendi. Hedef örnekleme hızı: {target_sampling_rate} Hz")

        # Ön işleme fonksiyonu
        def preprocess_function(batch):
            audio_arrays = []
            for audio_item in batch["audio"]:
                speech_array = audio_item["array"]
                current_sampling_rate = audio_item["sampling_rate"]

                if current_sampling_rate != target_sampling_rate:
                    speech_tensor = torch.tensor(speech_array).unsqueeze(0)
                    resampler = T.Resample(orig_freq=current_sampling_rate, new_freq=target_sampling_rate)
                    resampled_speech_tensor = resampler(speech_tensor)
                    speech_array_processed = resampled_speech_tensor.squeeze(0).numpy()
                else:
                    speech_array_processed = speech_array

                audio_arrays.append(speech_array_processed)

            # Feature extractor'ı uygula - DÜZELTİLMİŞ KISIM
            inputs = feature_extractor(
                audio_arrays,
                sampling_rate=target_sampling_rate,
                padding="max_length",    # <<<--- padding stratejisi max_length'e göre
                max_length=16000,        # <<<--- max_length tanımlandı (1 saniye için 16000 sample)
                truncation=True,         # <<<--- truncation şimdi max_length ile çalışacak
                return_tensors="pt"      # PyTorch tensörleri olarak döndür
            )
            return inputs

        print(f"\nSes veri seti ön işleniyor (örnekleme hızı {target_sampling_rate} Hz'e ayarlanıyor, max_length=16000)...")
        print("Bu işlem veri setinin büyüklüğüne göre biraz zaman alabilir.")

        global encoded_dataset # Diğer hücrelerde gerekebilir
        encoded_dataset = dataset_raw.map(
            preprocess_function,
            batched=True,
            batch_size=100
        )

        print(f"\nMap sonrası '{list(encoded_dataset.keys())[0]}' alt kümesinin sütunları: {encoded_dataset[list(encoded_dataset.keys())[0]].column_names}")

        # Gereksiz sütunları kaldıralım ve sadece gerekli olanları tutalım
        # 'input_values' ve 'attention_mask' feature extractor tarafından döndürülür. 'label' zaten var.
        final_columns_to_keep = ['input_values', 'attention_mask', 'label']

        # Her bir split için sütunları ayıkla
        for split_name in encoded_dataset.keys():
            current_columns = encoded_dataset[split_name].column_names
            cols_to_remove_final = [col for col in current_columns if col not in final_columns_to_keep]
            if cols_to_remove_final:
                encoded_dataset[split_name] = encoded_dataset[split_name].remove_columns(cols_to_remove_final)

        # set_format("torch") zaten return_tensors="pt" ile büyük ölçüde sağlanır,
        # ancak DataLoader ile uyumluluk için yine de çağrılabilir.
        # Eğer .map() zaten tensörler döndürüyorsa, bu satır bazen gereksiz olabilir veya
        # Dataset objesinin iç yapısını değiştirebilir. Şimdilik teyit için bırakalım.
        try:
            encoded_dataset.set_format("torch")
        except Exception as e_set_format:
            print(f"set_format('torch') sırasında uyarı/hata (genellikle sorun değil): {e_set_format}")


        print("\nSes veri seti başarıyla ön işlendi.")
        print("Ön işlenmiş veri seti yapısı:")
        print(encoded_dataset)

        if list(encoded_dataset.keys()):
            split_to_show = list(encoded_dataset.keys())[0]
            if len(encoded_dataset[split_to_show]) > 0:
                print(f"\nÖn işlenmiş '{split_to_show}' setinden ilk örnek:")
                example_processed = encoded_dataset[split_to_show][0]
                print(f"  Özellikler: {list(example_processed.keys())}")
                if 'input_values' in example_processed:
                    print(f"  input_values şekli: {example_processed['input_values'].shape}")
                if 'attention_mask' in example_processed:
                    print(f"  attention_mask şekli: {example_processed['attention_mask'].shape}")
                print(f"  label: {example_processed['label']}")
            else:
                print(f"'{split_to_show}' seti boş.")

    except Exception as e:
        print(f"\nFeature extractor yüklenirken veya ön işleme sırasında bir hata oluştu: {e}")
        import traceback
        print(traceback.format_exc())
        if 'feature_extractor' in globals(): del feature_extractor
        if 'encoded_dataset' in globals(): del encoded_dataset
else:
    print("Hata: 'dataset_raw' bulunamadığı için ön işleme başlatılamadı.")
    if 'feature_extractor' in globals(): del feature_extractor
    if 'encoded_dataset' in globals(): del encoded_dataset

In [ ]:
# Hücre 5: Wav2Vec2 Modelinin Yüklenmesi (Ses Sınıflandırması İçin)

from transformers import AutoModelForAudioClassification

# model_checkpoint_w2v2 Hücre 4'te tanımlanmıştı: "facebook/wav2vec2-base"
# label_feature Hücre 3'te tanımlanmıştı ve sınıf sayısını içeriyordu.
# device Hücre 2'de tanımlanmıştı.

if 'model_checkpoint_w2v2' in globals() and \
   'label_feature' in globals() and label_feature is not None and hasattr(label_feature, 'num_classes') and \
   'device' in globals():

    num_classes = label_feature.num_classes
    print(f"'{model_checkpoint_w2v2}' modeli '{num_classes}' etiket ile ses sınıflandırma görevi için yükleniyor...")

    try:
        # model_w2v2'yi global yapalım
        global model_w2v2
        model_w2v2 = AutoModelForAudioClassification.from_pretrained(
            model_checkpoint_w2v2,
            num_labels=num_classes,
            # Wav2Vec2, pooling modunu veya attention mask'in nasıl kullanılacağını belirten
            # bazı ek config parametreleri alabilir. Şimdilik varsayılanları kullanalım.
            # Eğer feature extractor 'attention_mask' döndürmediyse ve model bunu bekliyorsa,
            # config'de `m.config.apply_spec_augment = False` veya `m.config.mask_time_prob = 0` gibi ayarlar gerekebilir
            # ya da feature extractor'ın attention_mask döndürmesi sağlanabilir.
            # Şimdilik en basit haliyle yükleyelim.
            # ignore_mismatched_sizes=True, eğer önceden eğitilmiş başlık varsa ve bizim num_labels ile uyuşmuyorsa
            # yeniden başlatmayı zorlar. num_labels verdiğimizde bu genellikle otomatik yönetilir.
        )

        model_w2v2.to(device) # Modeli tanımlanan cihaza (GPU/CPU) taşı

        print(f"\nModel başarıyla yüklendi ve '{device}' cihazına taşındı.")

        total_params = sum(p.numel() for p in model_w2v2.parameters())
        trainable_params = sum(p.numel() for p in model_w2v2.parameters() if p.requires_grad)
        print(f"\nModeldeki toplam parametre sayısı: {total_params:,}")
        print(f"Modeldeki eğitilebilir parametre sayısı: {trainable_params:,}")

        # Modelin ve sınıflandırıcısının doğru sayıda etiket için yapılandırıldığını doğrula
        if hasattr(model_w2v2.config, 'num_labels'):
            print(f"Modelin yapılandırmasındaki etiket sayısı: {model_w2v2.config.num_labels}")
        # AutoModelForAudioClassification genellikle bir 'classifier' katmanına sahiptir.
        # Wav2Vec2ForSequenceClassification için bu katmanın adı 'classifier'dır.
        # Eğer farklı bir isimdeyse (örn: projection_head, head vb.), ona göre kontrol edin.
        # Genellikle son linear katmanın out_features'ı num_labels olmalıdır.
        # Wav2Vec2ForSequenceClassification'da bu model.classifier.out_features'dır.
        if hasattr(model_w2v2, 'classifier') and hasattr(model_w2v2.classifier, 'out_features'):
             print(f"Sınıflandırıcı katmanının ('classifier') çıktı boyutu: {model_w2v2.classifier.out_features}")
        elif hasattr(model_w2v2, 'projector') and hasattr(model_w2v2.projector, 'out_features'): # Bazen 'projector' olabilir
             print(f"Sınıflandırıcı katmanının ('projector') çıktı boyutu: {model_w2v2.projector.out_features}")
        else:
            # Son katmanı bulmaya çalışalım (genellikle en son nn.Linear)
            final_layer = None
            for _, module in reversed(list(model_w2v2.named_modules())):
                if isinstance(module, torch.nn.Linear):
                    final_layer = module
                    break
            if final_layer is not None:
                print(f"Modelin son Linear katmanının çıktı boyutu: {final_layer.out_features}")
            else:
                print("Sınıflandırıcı katmanının çıktı boyutu otomatik olarak tespit edilemedi, ancak num_labels doğru ayarlanmış olmalı.")


    except Exception as e:
        print(f"\nModel yüklenirken bir hata oluştu: {e}")
        import traceback
        print(traceback.format_exc())
        if 'model_w2v2' in globals(): del model_w2v2
else:
    print("Hata: Gerekli değişkenler ('model_checkpoint_w2v2', 'label_feature', 'device') bulunamadı.")
    if 'model_w2v2' in globals(): del model_w2v2

In [ ]:
# Hücre 6: Eğitim Argümanları, Metrikler ve Wav2Vec2 Modelinin Eğitilmesi

from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
# numpy, torch, time zaten Hücre 2'de import edilmişti.
# model_w2v2, encoded_dataset, feature_extractor, device değişkenlerinin
# önceki hücrelerde doğru şekilde tanımlandığını ve erişilebilir olduğunu varsayıyoruz.
# label_feature da Hücre 3'ten gelmeli (sınıf sayısı ve isimleri için).

# 1. Performans Metriklerini Hesaplama Fonksiyonu (Ses Sınıflandırması İçin)
def compute_metrics_audio(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Çok sınıflı durum için 'macro' veya 'weighted' ortalama kullanılabilir.
    # 'macro' her sınıfı eşit kabul eder, 'weighted' sınıf büyüklüğüne göre ağırlar.
    # Proje isterlerinde genel bir F1 beklentisi var, 'macro' F1 genellikle iyi bir metriktir.
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='macro', zero_division=0)
    acc = accuracy_score(labels, preds)

    # Specificity (Çok Sınıflı Durumda):
    # Her sınıf için One-vs-Rest mantığıyla TN/(TN+FP) hesaplayıp ortalamasını alabiliriz.
    # Ya da makro ortalamalı bir karmaşıklık matrisinden türetebiliriz.
    # Şimdilik, her sınıf için ayrı ayrı hesaplayıp ortalamasını alalım.
    num_classes_for_metrics = label_feature.num_classes if 'label_feature' in globals() and label_feature else 12 # Fallback
    conf_matrix = confusion_matrix(labels, preds, labels=list(range(num_classes_for_metrics)))
    per_class_specificity = []
    for i in range(num_classes_for_metrics):
        tn = np.sum(conf_matrix) - (np.sum(conf_matrix[i,:]) + np.sum(conf_matrix[:,i]) - conf_matrix[i,i])
        fp = np.sum(conf_matrix[:,i]) - conf_matrix[i,i]
        if (tn + fp) > 0:
            per_class_specificity.append(tn / (tn + fp))
        else:
            per_class_specificity.append(0.0)
    specificity_macro = np.mean(per_class_specificity) if per_class_specificity else 0.0

    # AUC (Çok Sınıflı Durumda):
    # One-vs-Rest (OvR) veya One-vs-One (OvO) stratejisiyle, genellikle 'macro' ortalamalı.
    auc_macro = 0.0
    if pred.predictions.ndim == 2 and pred.predictions.shape[1] == num_classes_for_metrics:
        try:
            logits_tensor = torch.tensor(pred.predictions)
            probs = torch.softmax(logits_tensor, dim=-1).cpu().numpy()
            # multi_class='ovr' (one-vs-rest) veya 'ovo' (one-vs-one)
            # average='macro' veya 'weighted'
            auc_macro = roc_auc_score(labels, probs, multi_class='ovr', average='macro')
        except ValueError as e: # Örneğin, bir sınıfa ait hiç örnek yoksa veya tek bir sınıf varsa
            # print(f"AUC (macro) hesaplanırken uyarı: {e}")
            auc_macro = 0.0

    return {
        'accuracy': acc,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro, # Sensitivity (macro)
        'f1_macro': f1_macro,
        'specificity_macro': specificity_macro,
        'auc_macro': auc_macro
    }

# 2. Eğitim Argümanları (TrainingArguments)
# Ses modelleri daha fazla VRAM tüketebilir, batch size'ı dikkatli ayarlayın.
training_args_w2v2 = TrainingArguments(
    output_dir="./wav2vec2_ks_results",      # Yeni bir çıktı dizini
    num_train_epochs=5,                      # Ses modelleri için biraz daha fazla epoch denenebilir (örn: 5-10)
    per_device_train_batch_size=8,           # Daha küçük batch size (VRAM'a göre)
    per_device_eval_batch_size=16,          # Değerlendirme için de
    learning_rate=3e-5,                      # Wav2Vec2 için yaygın bir öğrenme oranı
    warmup_ratio=0.1,                        # Isınma adımı oranı
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",        # Çok sınıflı için F1-macro iyi bir metriktir
    save_total_limit=2,
    report_to="tensorboard",
    # remove_unused_columns=False, # Eğer .map() sonrası gereksiz sütunlar kaldıysa ve Trainer hata veriyorsa True yapın
                                   # Ancak Hücre 4'te sütunları temizlemiştik.
    # fp16=torch.cuda.is_available(), # Karışık hassasiyetli eğitimi deneyebilirsiniz
)

print(f"\nEğitim argümanları (Wav2Vec2 için) tanımlandı. Çıktı dizini: {training_args_w2v2.output_dir}")

# 3. Trainer Objesinin Oluşturulması
# Gerekli değişkenlerin varlığını kontrol edelim
if 'model_w2v2' in globals() and model_w2v2 is not None and \
   'encoded_dataset' in globals() and encoded_dataset is not None and \
   'feature_extractor' in globals() and feature_extractor is not None and \
   'device' in globals() : # training_time burada henüz yok

    # trainer_w2v2'yi global yapalım
    global trainer_w2v2
    trainer_w2v2 = Trainer(
        model=model_w2v2,
        args=training_args_w2v2,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=feature_extractor, # Ses modelleri için tokenizer yerine feature_extractor verilir
        compute_metrics=compute_metrics_audio
    )
    print("\nTrainer objesi (Wav2Vec2 için) başarıyla oluşturuldu.")

    print("\nWav2Vec2 modeli eğitimi başlatılıyor...")
    print(f"Kullanılan cihaz: {device}. Epoch: {training_args_w2v2.num_train_epochs}, Train Batch: {training_args_w2v2.per_device_train_batch_size}")

    # training_time_w2v2'yi global yapalım
    global training_time_w2v2
    start_time_w2v2_train = time.time()
    try:
        train_result_w2v2 = trainer_w2v2.train()
        end_time_w2v2_train = time.time()
        training_time_w2v2 = end_time_w2v2_train - start_time_w2v2_train

        print(f"\nEğitim tamamlandı! Toplam eğitim süresi: {training_time_w2v2:.2f} saniye ({training_time_w2v2/60:.2f} dakika).")

        if hasattr(train_result_w2v2, 'metrics') and train_result_w2v2.metrics:
            print("Genel eğitim sonuç metrikleri (trainer.train() dönüşünden):")
            for key, value in train_result_w2v2.metrics.items():
                print(f"  {key}: {value}")

        best_model_path_w2v2 = f"{training_args_w2v2.output_dir}/best_model"
        trainer_w2v2.save_model(best_model_path_w2v2)
        # Feature extractor'ı da kaydetmek iyi bir pratiktir
        feature_extractor.save_pretrained(best_model_path_w2v2)
        print(f"Eğitim sonrası (en iyi) Wav2Vec2 modeli ve feature extractor '{best_model_path_w2v2}' adresine kaydedildi.")

    except Exception as e:
        print(f"\nWav2Vec2 modeli eğitimi sırasında bir hata oluştu: {e}")
        import traceback
        print(traceback.format_exc())
        if 'train_result_w2v2' not in globals(): train_result_w2v2 = None
        training_time_w2v2 = None
else:
    print("\nModel, işlenmiş veri seti veya feature extractor bulunamadığı için Trainer oluşturulamadı ve eğitim başlatılamadı.")
    if 'trainer_w2v2' in globals(): del trainer_w2v2
    if 'train_result_w2v2' not in globals(): train_result_w2v2 = None
    training_time_w2v2 = None

In [ ]:
# Hücre 7 (Wav2Vec2): Eğitilmiş Modelin Test Seti Üzerinde Değerlendirilmesi

print("Eğitilmiş Wav2Vec2 modeli test seti üzerinde değerlendiriliyor...")

# trainer_w2v2 ve encoded_dataset'in önceki hücrelerden doğru geldiğini varsayıyoruz.
if 'trainer_w2v2' in globals() and trainer_w2v2 is not None and \
   'encoded_dataset' in globals() and "test" in encoded_dataset:
    try:
        # Değerlendirilecek veri setini belirtiyoruz (test seti)
        # trainer_w2v2.model zaten en iyi modeli içermeli (load_best_model_at_end=True sayesinde)
        test_metrics_w2v2_output = trainer_w2v2.evaluate(eval_dataset=encoded_dataset["test"])

        print("\nWav2Vec2 Test Seti Performans Metrikleri:")
        # Trainer evaluate metodu, metrik isimlerinin başına 'eval_' ekler.
        print(f"  Test Kaybı (Loss): {test_metrics_w2v2_output.get('eval_loss', 'N/A'):.4f}")
        print(f"  Test Accuracy: {test_metrics_w2v2_output.get('eval_accuracy', 'N/A'):.4f}")
        print(f"  Test Precision (Macro): {test_metrics_w2v2_output.get('eval_precision_macro', 'N/A'):.4f}")
        print(f"  Test Recall (Macro): {test_metrics_w2v2_output.get('eval_recall_macro', 'N/A'):.4f}")
        print(f"  Test F1-Score (Macro): {test_metrics_w2v2_output.get('eval_f1_macro', 'N/A'):.4f}")
        print(f"  Test Specificity (Macro): {test_metrics_w2v2_output.get('eval_specificity_macro', 'N/A'):.4f}")
        print(f"  Test AUC (Macro): {test_metrics_w2v2_output.get('eval_auc_macro', 'N/A'):.4f}") # Burası yine 0.0 gelebilir

        # Metrikleri daha sonra kullanmak üzere saklayalım
        global wav2vec2_test_metrics
        wav2vec2_test_metrics = test_metrics_w2v2_output

    except Exception as e:
        print(f"\nTest seti değerlendirmesi sırasında bir hata oluştu: {e}")
        import traceback
        print(traceback.format_exc())
        wav2vec2_test_metrics = None
else:
    print("Hata: 'trainer_w2v2' objesi veya 'encoded_dataset['test']' bulunamadı.")
    print("Lütfen önceki hücrelerin doğru çalıştığından emin olun.")
    wav2vec2_test_metrics = None

In [ ]:
# Hücre 8 (Wav2Vec2): Karmaşıklık Matrisi ve ROC Eğrisinin Çizdirilmesi

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, RocCurveDisplay
# sklearn.metrics.auc zaten Hücre 7'de compute_metrics_audio içinde roc_auc_score olarak kullanıldı.
# Burada RocCurveDisplay'i de kullanabiliriz.
import numpy as np
import torch
from sklearn.preprocessing import label_binarize # Çok sınıflı ROC için

# Gerekli değişkenlerin (trainer_w2v2, encoded_dataset, wav2vec2_test_metrics, label_feature)
# önceki hücrelerden doğru geldiğini varsayıyoruz.
if 'trainer_w2v2' in globals() and trainer_w2v2 is not None and \
   'encoded_dataset' in globals() and "test" in encoded_dataset and \
   'wav2vec2_test_metrics' in globals() and wav2vec2_test_metrics is not None and \
   'label_feature' in globals() and label_feature is not None and hasattr(label_feature, 'names'):

    print("Test seti üzerinde tahminler (logitler ve olasılıklar) alınıyor...")
    try:
        test_predictions_output = trainer_w2v2.predict(encoded_dataset["test"])

        logits = test_predictions_output.predictions
        probabilities = torch.softmax(torch.tensor(logits), dim=-1).cpu().numpy()
        predicted_labels = np.argmax(logits, axis=1)
        true_labels = test_predictions_output.label_ids

        num_classes = label_feature.num_classes
        class_names = label_feature.names

        # 1. Karmaşıklık Matrisi (Confusion Matrix)
        cm = confusion_matrix(true_labels, predicted_labels, labels=list(range(num_classes)))

        plt.figure(figsize=(10, 8)) # Matris daha büyük olacağı için boyutu ayarlayalım
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=class_names,
                    yticklabels=class_names)
        plt.title('Karmaşıklık Matrisi (Wav2Vec2 - Test Seti)')
        plt.xlabel('Tahmin Edilen Etiket')
        plt.ylabel('Gerçek Etiket')
        plt.xticks(rotation=45, ha="right")
        plt.yticks(rotation=0)
        plt.tight_layout() # Etiketlerin sıkışmasını önlemek için
        plt.show()

        # TN, FP, FN, TP değerlerini çok sınıflı durumda her sınıf için veya genel olarak vermek yerine,
        # karmaşıklık matrisinin kendisi ana bilgiyi verir.
        # İstenirse her sınıf için bu değerler ayrıca hesaplanabilir.

        # 2. ROC Eğrisi ve AUC (Çok Sınıflı Durum - Macro Average)
        # Gerçek etiketleri binarize et (one-hot encoding gibi)
        y_true_binarized = label_binarize(true_labels, classes=list(range(num_classes)))

        # Her sınıf için ROC eğrisi çizdirme ve makro-ortalama AUC hesaplama
        fpr = dict()
        tpr = dict()
        roc_auc = dict()

        for i in range(num_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], probabilities[:, i])
            roc_auc[i] = sklearn.metrics.auc(fpr[i], tpr[i])

        # Makro-ortalama ROC Eğrisi
        # Öncelikle tüm fpr değerlerini birleştirelim (interpolasyon gerekli olabilir)
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
        # Sonra her bir ROC eğrisini bu noktalara göre interpolasyon yapalım
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(num_classes):
            mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
        # Ortalamasını alalım
        mean_tpr /= num_classes
        roc_auc_macro_manual = sklearn.metrics.auc(all_fpr, mean_tpr)

        plt.figure(figsize=(10, 8))
        plt.plot(all_fpr, mean_tpr,
                 label=f'Makro-Ortalama ROC eğrisi (AUC = {roc_auc_macro_manual:.4f})',
                 color='deeppink', linestyle=':', linewidth=4)

        # Her sınıf için ROC eğrisini de çizdirebiliriz (isteğe bağlı, grafik kalabalıklaşabilir)
        # from itertools import cycle
        # colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'magenta'])
        # for i, color in zip(range(num_classes), colors):
        #     plt.plot(fpr[i], tpr[i], color=color, lw=1.5, alpha=0.7,
        #              label=f'ROC Sınıf {class_names[i]} (AUC = {roc_auc[i]:.2f})')

        plt.plot([0, 1], [0, 1], 'k--', lw=2) # Rastgele tahmin çizgisi
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Yanlış Pozitif Oranı (False Positive Rate)')
        plt.ylabel('Doğru Pozitif Oranı (True Positive Rate)')
        plt.title('Çok Sınıflı Alıcı İşletim Karakteristiği (ROC) Eğrisi - Makro Ortalama')
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.show()

        print(f"\nHücre 7'de hesaplanan Test AUC (Macro) (trainer.evaluate): {wav2vec2_test_metrics.get('eval_auc_macro', 'N/A'):.4f}")
        print(f"Bu hücrede Makro-Ortalama ROC için hesaplanan AUC (sklearn): {roc_auc_macro_manual:.4f}")
        print("Not: İki AUC değerinin yakın olması beklenir. Farklılıklar, sklearn'deki roc_auc_score'un 'ovr' stratejisi ile")
        print("burada çizilen manuel makro-ortalama ROC eğrisi arasındaki hesaplama detaylarından kaynaklanabilir.")

    except Exception as e:
        print(f"\nTahminler alınırken veya görselleştirmeler oluşturulurken bir hata oluştu: {e}")
        import traceback
        print(traceback.format_exc())
else:
    print("Hata: Gerekli değişkenler ('trainer_w2v2', 'encoded_dataset['test']', 'wav2vec2_test_metrics', 'label_feature') bulunamadı.")
    print("Lütfen önceki hücrelerin doğru çalıştığından emin olun.")

In [ ]:
# Hücre 9 (Wav2Vec2): Eğitim ve Doğrulama Kayıp/Metrik Grafikleri

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# trainer_w2v2 ve log_history'nin Hücre 6'dan geldiğini varsayıyoruz
if 'trainer_w2v2' in globals() and trainer_w2v2 is not None and \
   hasattr(trainer_w2v2.state, 'log_history') and trainer_w2v2.state.log_history:

    log_history_w2v2 = trainer_w2v2.state.log_history

    print("Wav2Vec2 Ham log_history içeriği (ilk 3 ve son 3 kayıt - eğer yeterliyse):")
    num_entries_to_show = 3
    if len(log_history_w2v2) > 2 * num_entries_to_show:
        for i in range(num_entries_to_show): print(log_history_w2v2[i])
        print("...")
        for i in range(len(log_history_w2v2) - num_entries_to_show, len(log_history_w2v2)): print(log_history_w2v2[i])
    else:
        for entry in log_history_w2v2: print(entry)
    print("-" * 50)

    log_df_w2v2 = pd.DataFrame(log_history_w2v2)

    # Eğitim logları (sadece 'loss' içeren ve 'eval_loss' içermeyenler)
    train_logs_df_w2v2 = log_df_w2v2[log_df_w2v2['loss'].notna() & log_df_w2v2['eval_loss'].isna()].copy()

    # Değerlendirme logları ('eval_loss' içerenler)
    eval_logs_df_w2v2 = log_df_w2v2[log_df_w2v2['eval_loss'].notna()].copy()

    # Grafikleri çizdirelim
    if not train_logs_df_w2v2.empty and not eval_logs_df_w2v2.empty:
        # --- Kayıp Grafiği ---
        plt.figure(figsize=(12, 6))
        plt.plot(train_logs_df_w2v2['epoch'], train_logs_df_w2v2['loss'], 'b-o', label='Eğitim Kaybı (Training Loss)')
        plt.plot(eval_logs_df_w2v2['epoch'], eval_logs_df_w2v2['eval_loss'], 'r-s', label='Doğrulama Kaybı (Validation Loss)')
        plt.title('Wav2Vec2: Epoch Bazında Eğitim ve Doğrulama Kaybı')
        plt.xlabel('Epoch')
        plt.ylabel('Kayıp (Loss)')
        plt.legend()
        plt.grid(True)
        epochs_present = sorted(list(set(eval_logs_df_w2v2['epoch'].round().tolist())))
        if epochs_present:
            min_e, max_e = min(epochs_present), max(epochs_present)
            plt.xticks(np.arange(int(min_e), int(max_e) + 1, 1.0))
            plt.xlim(left=int(min_e) - 0.5 if int(min_e) > 0 else 0, right=int(max_e) + 0.5)
        plt.show()

        # --- Doğruluk ve F1 Macro Grafiği ---
        fig, ax1 = plt.subplots(figsize=(12, 6))

        # Sol y-ekseni (Doğruluk)
        color = 'tab:green'
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Doğruluk (Accuracy)', color=color)
        ax1.plot(eval_logs_df_w2v2['epoch'], eval_logs_df_w2v2['eval_accuracy'], color=color, marker='^', linestyle='-', label='Doğrulama Doğruluğu')
        ax1.tick_params(axis='y', labelcolor=color)

        # Sağ y-ekseni (F1 Macro)
        ax2 = ax1.twinx() # Aynı x-eksenini paylaşan ikinci bir y-ekseni
        color = 'tab:purple'
        ax2.set_ylabel('F1 Skoru (Macro)', color=color)
        ax2.plot(eval_logs_df_w2v2['epoch'], eval_logs_df_w2v2['eval_f1_macro'], color=color, marker='x', linestyle='--', label='Doğrulama F1 (Macro)')
        ax2.tick_params(axis='y', labelcolor=color)

        plt.title('Wav2Vec2: Epoch Bazında Doğrulama Metrikleri (Accuracy & F1 Macro)')
        fig.tight_layout() # Etiketlerin üst üste binmemesi için
        # Lejantları birleştirmek için:
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc='best')

        if epochs_present: # Aynı x-ekseni ayarını kullanalım
            plt.xticks(np.arange(int(min_e), int(max_e) + 1, 1.0))
            ax1.set_xlim(left=int(min_e) - 0.5 if int(min_e) > 0 else 0, right=int(max_e) + 0.5)

        plt.grid(True)
        plt.show()

    else:
        print("Grafik çizdirmek için yeterli eğitim veya doğrulama logu bulunamadı.")
        if train_logs_df_w2v2.empty: print("Eğitim logları filtrelenemedi.")
        if eval_logs_df_w2v2.empty: print("Doğrulama logları filtrelenemedi.")
else:
    print("Hata: 'trainer_w2v2' objesi veya 'log_history' bulunamadı.")

In [ ]:
# Hücre 10 (Wav2Vec2): Eğitim ve Çıkarım Sürelerinin Hesaplanması

# time, torch, tqdm.auto, DataLoader zaten import edilmiş olmalı (Hücre 2'den veya önceki hücrelerden)
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

# 1. Eğitim Süresi
calculated_training_time_w2v2 = None
if 'training_time_w2v2' in globals() and training_time_w2v2 is not None: # Hücre 6'da global yapılmıştı
    calculated_training_time_w2v2 = training_time_w2v2
elif 'trainer_w2v2' in globals() and trainer_w2v2 is not None and \
     hasattr(trainer_w2v2.state, 'log_history') and trainer_w2v2.state.log_history:
    for log_entry in reversed(trainer_w2v2.state.log_history):
        if 'train_runtime' in log_entry:
            calculated_training_time_w2v2 = log_entry['train_runtime']
            break

if calculated_training_time_w2v2 is not None:
    print(f"Toplam Wav2Vec2 Model Eğitim Süresi: {calculated_training_time_w2v2:.2f} saniye ({calculated_training_time_w2v2/60:.2f} dakika)")
else:
    print("Wav2Vec2 eğitim süresi bilgisi `training_time_w2v2` değişkeninden veya `log_history`'den alınamadı.")

# 2. Örnek Başına Ortalama Çıkarım Süresi (Test Seti Üzerinden)
avg_inference_time_per_sample_w2v2 = None
# Gerekli değişkenlerin varlığını kontrol edelim
if 'model_w2v2' in globals() and model_w2v2 is not None and \
   'encoded_dataset' in globals() and "test" in encoded_dataset and \
   'device' in globals() and 'feature_extractor' in globals() and feature_extractor is not None :

    model_w2v2.eval() # Modeli değerlendirme moduna al

    # Modelin beklediği giriş sütunlarını belirle
    # Wav2Vec2FeatureExtractor genellikle 'input_values' ve 'attention_mask' (eğer padding varsa) döndürür
    model_input_names_w2v2 = []
    if 'input_values' in encoded_dataset["test"].features:
        model_input_names_w2v2.append('input_values')
    if 'attention_mask' in encoded_dataset["test"].features: # Hücre 4'te attention_mask eklenmemişti
        model_input_names_w2v2.append('attention_mask')

    if not model_input_names_w2v2: # Eğer hiçbir şey bulamazsak, varsayılanı kullanalım
         model_input_names_w2v2 = ['input_values']
         print(f"Uyarı: Model giriş sütunları otomatik algılanamadı, varsayılan kullanılıyor: {model_input_names_w2v2}")


    try:
        # Sadece modelin ihtiyaç duyduğu sütunları içeren bir test seti kopyası oluştur
        inference_ready_dataset_w2v2 = encoded_dataset["test"].select_columns(model_input_names_w2v2)
        inference_ready_dataset_w2v2.set_format("torch")
    except Exception as e_cols:
        print(f"Uyarı: Çıkarım için sütun seçimi/formatlamada sorun ({e_cols}). Orijinal test seti kullanılacak.")
        inference_ready_dataset_w2v2 = encoded_dataset["test"]


    inference_batch_size_w2v2 = training_args_w2v2.per_device_eval_batch_size if 'training_args_w2v2' in globals() else 16
    inference_dataloader_w2v2 = DataLoader(inference_ready_dataset_w2v2, batch_size=inference_batch_size_w2v2)

    total_inference_time_val_w2v2 = 0.0
    num_samples_processed_w2v2 = 0

    print(f"\nWav2Vec2 Test seti üzerinde çıkarım süresi hesaplanıyor (Batch Size: {inference_batch_size_w2v2})...")

    with torch.no_grad():
        for batch in tqdm(inference_dataloader_w2v2, desc="Wav2Vec2 Çıkarım"):
            inputs_for_model = {}
            valid_batch = True
            for name in model_input_names_w2v2:
                if name in batch:
                    inputs_for_model[name] = batch[name].to(device)
                else:
                    if name == 'input_values': valid_batch = False
                    break

            if not valid_batch or not inputs_for_model :
                try: num_in_batch = len(batch.get(model_input_names_w2v2[0], [])) if batch and model_input_names_w2v2 else 0
                except: num_in_batch = 0
                if num_in_batch > 0 : print(f"Uyarı: Geçerli model girdisi eksik, {num_in_batch} örnek içeren bu yığın atlandı.")
                continue

            current_batch_size = len(inputs_for_model[model_input_names_w2v2[0]]) # İlk ana girdi sütununun boyutu
            num_samples_processed_w2v2 += current_batch_size

            start_batch_time = time.perf_counter()
            outputs = model_w2v2(**inputs_for_model)
            end_batch_time = time.perf_counter()

            batch_time = end_batch_time - start_batch_time
            total_inference_time_val_w2v2 += batch_time

    if num_samples_processed_w2v2 > 0 and total_inference_time_val_w2v2 > 0 :
        avg_inference_time_per_sample_w2v2 = total_inference_time_val_w2v2 / num_samples_processed_w2v2
        samples_per_second_inference_w2v2 = num_samples_processed_w2v2 / total_inference_time_val_w2v2
        print(f"\nToplam {num_samples_processed_w2v2} örnek için Wav2Vec2 çıkarım süresi: {total_inference_time_val_w2v2:.4f} saniye")
        print(f"Örnek başına ortalama Wav2Vec2 çıkarım süresi: {avg_inference_time_per_sample_w2v2:.6f} saniye/örnek")
        print(f"Saniyede işlenen Wav2Vec2 örnek sayısı (çıkarım): {samples_per_second_inference_w2v2:.2f} örnek/saniye")
    elif num_samples_processed_w2v2 > 0 and total_inference_time_val_w2v2 == 0 :
        print(f"\nToplam {num_samples_processed_w2v2} örnek için Wav2Vec2 çıkarım süresi çok kısa, hız çok yüksek.")
        avg_inference_time_per_sample_w2v2 = 0.0
    else:
        print("Wav2Vec2 Çıkarım için hiç örnek işlenemedi veya süre ölçülemedi.")

else:
    print("Hata: Wav2Vec2 Modeli, işlenmiş test seti, feature extractor veya cihaz bilgisi bulunamadı.")